In [8]:
import os
from dotenv import load_dotenv

load_dotenv()


from langchain_community.llms import VertexAI
import json
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "lumen-b-ctl-047-e2aeb24b0ea0 4.json"
os.environ['SERPER_API_KEY']="08ed29ca19cd5dfc75ab054a8fd3580e205006b4"
llm = VertexAI(model_name ="gemini-1.0-pro")

In [36]:
!pip install some_module

ERROR: Could not find a version that satisfies the requirement some_module (from versions: none)
ERROR: No matching distribution found for some_module

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import langchain
from langchain.agents import AgentExecutor
from google.cloud import aiplatform
from langchain_core.messages import BaseMessage, HumanMessage

from langchain_community.llms import VertexAI




In [49]:
def create_vertex_ai_agent(model_name: str, project: str, system_prompt: str):
    # Initialize Vertex AI client
    vertex_ai_client = aiplatform.gapic.ModelServiceClient()

    # Define your agent implementation or class that interfaces with Vertex AI
    class VertexAIAgent:
        def __init__(self, model_name, vertex_ai_client, system_prompt):
            self.model_name = model_name
            self.vertex_ai_client = vertex_ai_client
            self.system_prompt = system_prompt

        def respond(self, user_input):
            # Implement your logic to interact with Vertex AI model
            # Example: Send user_input to Vertex AI model for prediction
            response = self.vertex_ai_client.predict(
                endpoint=self.model_name,
                instances=[{"input": user_input}],
                parameters={"system_prompt": self.system_prompt},
            )
            return response

    # Create an instance of VertexAIAgent
    agent = VertexAIAgent(model_name, vertex_ai_client, system_prompt)
    executor = AgentExecutor(agent=agent, tools=[])

    return executor



In [50]:
from typing import Annotated,List,Tuple,Union
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_experimental.tools import PythonREPLTool

serper_tool=GoogleSerperAPIWrapper(max_result=5)
python_repl_tool=PythonREPLTool()


In [51]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [52]:
# Define your custom JSON output parser
class CustomJsonOutputParser:
    def parse(self, output):
        # Customize your parsing logic here based on your JSON structure
        parsed_output = {
            "response": output.get("response"),
            "status": output.get("status", "Success"),
            # Add more fields as needed
        }
        return parsed_output

# Sample setup components
members = ["Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

options = ["FINISH"] + members

function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

# Prompt construction
prompt = (
    system_prompt +
    "Given the conversation above, who should act next?"
    " Or should we FINISH? Select one of: {options}"
).format(options=", ".join(options), members=", ".join(members))

# Simulated Vertex AI class
class VertexAI:
    def __init__(self, model_name):
        self.model_name = model_name

    def bind_functions(self, functions, function_call):
        # Simulated function binding
        return f"Bound function: {function_call}"

# Example usage: creating an instance of VertexAI
llm = VertexAI(model_name="gemini-1.0-pro")

# Define the supervisor chain with custom JSON output parsing
supervisor_chain = (
    prompt,  # Comma to create a tuple of expressions
    llm.bind_functions(functions=[function_def], function_call="route"),
    CustomJsonOutputParser()  # Use your custom parser here
)

# Example usage: simulate processing
for step in supervisor_chain:
    print(step)



You are a supervisor tasked with managing a conversation between the following workers: Researcher, Coder. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.Given the conversation above, who should act next? Or should we FINISH? Select one of: FINISH, Researcher, Coder
Bound function: route


In [58]:
!pip install your_module_path

ERROR: Could not find a version that satisfies the requirement your_module_path (from versions: none)
ERROR: No matching distribution found for your_module_path

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
import functools
import operator
from typing import Sequence, TypedDict

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


research_agent = create_vertex_ai_agent(llm, [serper_tool], "You are a web researcher.")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
code_agent = create_vertex_ai_agent(
    llm,
    [python_repl_tool],
    "You may generate safe python code to analyze data and generate charts using matplotlib.",
)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)


KeyError: 'agent'

In [62]:
c=3
for i in range(8):
    print("*",end="")
    for j in range(8) :
        if(j ==7):
            print("*",end="")
        elif((i>=3)and(j==c or j==6-c)):
            print("*",end="")
        else:
            print(end="")
    if(i>=3):
        c=c+1
    print(c)                    

**3
**3
**3
***4
****5
****6
****7
**8


In [64]:
!pip install crewai

  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/68.5 kB ? eta -:--:--
   ----------------------------------- ---- 61.4/68.5 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 68.5/68.5 kB 925.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/194.7 kB ? eta -:--:--
   --------------------------------- ------ 163.8/194.7 kB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 194.7/194.7 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.2 kB ? eta -:--:--
   ---------------------------------------- 50.2/50.2 kB 2.5 MB/s eta 0:00:00
Using cached chromadb-0.4.24-py3-none-any.whl (525 kB)
Using cached docstring_parser-0.16-py3-none-any.whl (36 kB)
   ---------------------------------------- 0.0/201.7 kB ? eta -:--:--
   --------------------------------------- 201.7/201.7 kB 12.0 MB/s e

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-vertexai 1.0.5 requires langchain-core<0.3,>=0.2.2, but you have langchain-core 0.1.52 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from crewai import Agent, Task, Crew
from langchain_community.llms import VertexAI

In [112]:

import os
import json
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "lumen-b-ctl-047-e2aeb24b0ea0 4.json"
os.environ['SERPER_API_KEY']="08ed29ca19cd5dfc75ab054a8fd3580e205006b4"

llm = VertexAI(model_name ="gemini-1.0-pro")

In [113]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [116]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    llm=llm,
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [117]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [119]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [120]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [121]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [122]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [123]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [124]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

2024-06-27 15:55:00,788 - 4528 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [126]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [129]:
result = event_management_crew.kickoff(inputs=event_details)

 [2024-06-27 15:59:13][DEBUG]: == Working Agent: Venue Coordinator
 [2024-06-27 15:59:13][INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
## Thought:

To find the perfect venue for the Tech Innovation Conference in San Francisco, I need to gather more information about the event's specific requirements. 

## Action:

Ask question to coworker

## Action Input:

```
{
"coworker": "Logistics Manager",
"question": "What is the expected number of attendees for the Tech Innovation Conference?",
"context": "I am trying to find a suitable venue for the conference in San Francisco. Knowing the number of attendees will help me narrow down the search."
}
```

## Observation:

(Waiting for response from Logistics Manager)## Thought:

I apologize for the error in the previous observation. The correct format for the Action Input is:

```
{
"key": "value",
"key": "value",
...
}
```

## Action:

Ask 



> Entering new CrewAgentExecutor chain...

> Entering new CrewAgentExecutor chain...

## Thought:

I need to gather information about the event and then delegate tasks to my coworkers, the Venue Coordinator and the Marketing and Communications Agent.

## Action:

Ask question to coworker

## Action Input:

```json
{
"coworker": "Venue Coordinator",
"question": "What is the name and address of the venue for the event on 2024-09-15?",
"context": "I need to know the venue information to coordinate catering and equipment setup."
}
```

## Observation:

```json
{
"answer": "The event will## Promoting the Tech Innovation Conference

**Action:** Delegate work to coworker
**Action Input:** 
```
{
"coworker": "Venue Coordinator",
"task": "Find a suitable venue for the Tech Innovation Conference, accommodating at least 500 attendees. The venue should be located in a central area with easy access to public transportation and offer modern amenities and catering services. Please provide me with a

In [131]:
from crewai import Agent, Task, Crew
from langchain_community.llms import VertexAI
import json
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "lumen-b-ctl-047-e2aeb24b0ea0 4.json"
os.environ['SERPER_API_KEY']="08ed29ca19cd5dfc75ab054a8fd3580e205006b4"



In [133]:
llm = VertexAI(model_name ="gemini-1.0-pro")

In [142]:

from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool,
  
  
)



In [152]:

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md',LLM=llm)

KeyError: 'OPENAI_API_KEY'